In [2]:
import yfinance as yf

In [3]:
from datetime import datetime
end = datetime.now()
start = datetime(end.year-20, end.month, end.day)

In [4]:
stock = "GOOG"
google_data = yf.download(stock, start, end)

[*********************100%%**********************]  1 of 1 completed


In [5]:
import numpy as np

# Tính toán giá trị trung bình của mỗi ngày (giả định đơn giản)
google_data['Average'] = (google_data['High'] + google_data['Low']) / 2

# Chuẩn hóa dữ liệu
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(google_data[['Average']])

# Tạo cửa sổ dữ liệu
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data_scaled, seq_length)


In [6]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(seq_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [26]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=30, verbose=1, validation_data=(X_test, y_test))

Epoch 1/30
109/109 [==============================] - 0s 4ms/step - loss: 9.1994e-05 - val_loss: 9.7753e-05
Epoch 2/30
109/109 [==============================] - 0s 3ms/step - loss: 9.0491e-05 - val_loss: 7.8617e-05
Epoch 3/30
109/109 [==============================] - 1s 5ms/step - loss: 9.1449e-05 - val_loss: 8.5478e-05
Epoch 4/30
109/109 [==============================] - 1s 6ms/step - loss: 1.0337e-04 - val_loss: 8.2275e-05
Epoch 5/30
109/109 [==============================] - 1s 6ms/step - loss: 8.5143e-05 - val_loss: 7.4511e-05
Epoch 6/30
109/109 [==============================] - 1s 6ms/step - loss: 8.5504e-05 - val_loss: 7.5324e-05
Epoch 7/30
109/109 [==============================] - 1s 8ms/step - loss: 9.7850e-05 - val_loss: 8.3219e-05
Epoch 8/30
109/109 [==============================] - 1s 5ms/step - loss: 1.0101e-04 - val_loss: 8.5021e-05
Epoch 9/30
109/109 [==============================] - 0s 4ms/step - loss: 9.0688e-05 - val_loss: 1.1201e-04
Epoch 10/30
109/109 [=======

In [28]:
predictions = model.predict(X_test)

47/47 [==============================] - 0s 2ms/step


In [29]:
# Giả sử y_test là mảng chứa giá trị thực và predictions là kết quả dự đoán từ mô hình
accuracy_threshold = 0.05  # 5%
correct_predictions = np.abs(predictions.flatten() - y_test) <= accuracy_threshold * y_test
accuracy = np.mean(correct_predictions)
print(f"Độ chính xác trong ngưỡng {accuracy_threshold*100}%: {accuracy*100:.2f}%")

Độ chính xác trong ngưỡng 5.0%: 3.52%
